In [15]:
import json
import csv

# Función para leer datos meteorológicos desde un archivo JSON
def leer_datos_meteorologicos(nombre_archivo):
    with open(nombre_archivo, 'r') as archivo:
        datos = json.load(archivo)
    return datos

# Función para etiquetar la presencia de una ola de calor basada en datos climáticos
def etiquetar_ola_de_calor(datos_clima, umbral_temperatura):
    etiquetas = []
    for registro in datos_clima['data']['list']:
        temperatura = registro['main']['temp']
        if temperatura > umbral_temperatura:
            etiquetas.append(1)  # Hay ola de calor
        else:
            etiquetas.append(0)  # No hay ola de calor
    return etiquetas

# Función para cargar datos de instalaciones desde un archivo CSV y filtrar por tipo de instalación
def cargar_datos_instalaciones(nombre_archivo_csv, tipo_instalacion):
    csv.field_size_limit(500 * 1024)
    datos_instalaciones = []
    with open(nombre_archivo_csv, 'r', encoding='utf-8',newline='') as archivo_csv:
        lector_csv = csv.DictReader(archivo_csv)
        for fila in lector_csv:
            if fila['TIPO'] == tipo_instalacion:
                datos_instalaciones.append(fila)
    return datos_instalaciones

def obtener_recomendaciones(datos_clima, etiquetas_ola_calor, datos_instalaciones, tiene_problemas_salud):
    # Implementa la lógica para generar las recomendaciones de deportes
    # basadas en las condiciones climáticas, la ubicación del usuario y los datos
    # de instalaciones deportivas disponibles.

    recomendaciones = []

    # Implementa la lógica para recomendar deportes seguros según si hay ola de calor o no.
    if etiquetas_ola_calor[0] == 1:  # Primer registro en los datos de etiquetas corresponde a la predicción actual
        # Recomendar actividades en centros de salud si tiene problemas de salud
        if tiene_problemas_salud:
            recomendaciones.extend(obtener_recomendaciones_centros_salud(datos_instalaciones))
    else:
        # Recomendar actividades al aire libre en parques y jardines
        recomendaciones.extend(obtener_recomendaciones_parques(datos_instalaciones))

    return recomendaciones

# Función para obtener recomendaciones específicas para parques y jardines
def obtener_recomendaciones_parques(instalaciones):
    recomendaciones_parques = []
    for instalacion in instalaciones:
        if instalacion['TIPO'] == 'parque':
            recomendaciones_parques.append(f"Hacer ejercicio en el parque: {instalacion['NOMBRE']} ({instalacion['DIRECCION']})")
    return recomendaciones_parques

# Función para obtener recomendaciones específicas para centros de salud
def obtener_recomendaciones_centros_salud(instalaciones):
    recomendaciones_centros_salud = []
    for instalacion in instalaciones:
        if instalacion['TIPO'] == 'centro_salud':
            recomendaciones_centros_salud.append(f"Realizar actividad en centro de salud: {instalacion['NOMBRE']} ({instalacion['DIRECCION']})")
    return recomendaciones_centros_salud



if __name__ == "__main__":
    nombre_archivo_json = "../Raw_Datasets/Trabajado/tiempo.json"  # Reemplaza con el nombre de tu archivo JSON
    nombre_archivo_sitios = "../Raw_Datasets/items_Giacomo/items.csv"  # Reemplaza con el nombre de tu archivo CSV de sitios (piscinas, parques y polideportivos)

    umbral_temperatura_ola_calor = 30.0  # Reemplaza con el umbral de temperatura para definir una ola de calor
    tiene_problemas_salud = True  # Reemplaza con True si el usuario tiene problemas de salud, False si no

    datos_clima = leer_datos_meteorologicos(nombre_archivo_json)
    etiquetas_ola_calor = etiquetar_ola_de_calor(datos_clima, umbral_temperatura_ola_calor)

    datos_instalaciones = cargar_datos_instalaciones(nombre_archivo_sitios, 'piscina') + \
                          cargar_datos_instalaciones(nombre_archivo_sitios, 'parque') + \
                          cargar_datos_instalaciones(nombre_archivo_sitios, 'polideportivo')

    recomendaciones_deportes = obtener_recomendaciones(
        datos_clima, etiquetas_ola_calor, datos_instalaciones, tiene_problemas_salud
    )

    # Imprime las recomendaciones
    print("Recomendaciones de deportes:")
    for deporte in recomendaciones_deportes:
        print(deporte)


Recomendaciones de deportes:


In [17]:
import pandas as pd

In [24]:
f = pd.read_csv('../Raw_Datasets/items_Giacomo/items.csv',sep=',')
f['TIPO'].unique()

array(['FUENTE', 'INSTALACIÓN DEPORTIVA', 'PISCINA', 'ZONA VERDE',
       'SENDA', 'CENTRO DE SALUD'], dtype=object)

In [25]:
import json
import csv

# Función para leer datos meteorológicos desde un archivo JSON
def leer_datos_meteorologicos(nombre_archivo):
    with open(nombre_archivo, 'r') as archivo:
        datos = json.load(archivo)
    return datos

# Función para etiquetar la presencia de una ola de calor basada en datos climáticos
def etiquetar_ola_de_calor(datos_clima, umbral_temperatura):
    etiquetas = []
    for registro in datos_clima['data']['list']:
        temperatura = registro['main']['temp']
        if temperatura > umbral_temperatura:
            etiquetas.append(1)  # Hay ola de calor
        else:
            etiquetas.append(0)  # No hay ola de calor
    return etiquetas

# Función para cargar datos de instalaciones desde un archivo CSV y filtrar por tipo de instalación
def cargar_datos_instalaciones(nombre_archivo_csv, tipo_instalacion):
    csv.field_size_limit(500 * 1024)
    datos_instalaciones = []
    with open(nombre_archivo_csv, 'r', encoding='utf-8',newline='') as archivo_csv:
        lector_csv = csv.DictReader(archivo_csv)
        for fila in lector_csv:
            if fila['TIPO'] == tipo_instalacion:
                datos_instalaciones.append(fila)
    return datos_instalaciones

def obtener_recomendaciones(datos_clima, etiquetas_ola_calor, datos_instalaciones, tiene_problemas_salud):
    # Implementa la lógica para generar las recomendaciones de deportes
    # basadas en las condiciones climáticas, la ubicación del usuario y los datos
    # de instalaciones deportivas disponibles.

    recomendaciones = []

    # Implementa la lógica para recomendar deportes seguros según si hay ola de calor o no.
    if etiquetas_ola_calor[0] == 1:  # Primer registro en los datos de etiquetas corresponde a la predicción actual
        # Recomendar actividades en centros de salud si tiene problemas de salud
        if tiene_problemas_salud:
            recomendaciones.extend(obtener_recomendaciones_centros_salud(datos_instalaciones))
    else:
        # Recomendar actividades al aire libre en parques y jardines
        recomendaciones.extend(obtener_recomendaciones_parques(datos_instalaciones))

    return recomendaciones

# Función para obtener recomendaciones específicas para parques y jardines
def obtener_recomendaciones_parques(instalaciones):
    recomendaciones_parques = []
    for instalacion in instalaciones:
        if instalacion['TIPO'] == 'parque':
            recomendaciones_parques.append(f"Hacer ejercicio en el parque: {instalacion['NOMBRE']} ({instalacion['DIRECCION']})")
    return recomendaciones_parques

# Función para obtener recomendaciones específicas para centros de salud
def obtener_recomendaciones_centros_salud(instalaciones):
    recomendaciones_centros_salud = []
    for instalacion in instalaciones:
        if instalacion['TIPO'] == 'centro_salud':
            recomendaciones_centros_salud.append(f"Realizar actividad en centro de salud: {instalacion['NOMBRE']} ({instalacion['DIRECCION']})")
    return recomendaciones_centros_salud

if __name__ == "__main__":
    nombre_archivo_json = "../Raw_Datasets/Trabajado/tiempo.json"  # Reemplaza con el nombre de tu archivo JSON
    nombre_archivo_sitios = "../Raw_Datasets/items_Giacomo/items.csv"  # Reemplaza con el nombre de tu archivo CSV de sitios (piscinas, parques y polideportivos)

    umbral_temperatura_ola_calor = 30.0  # Reemplaza con el umbral de temperatura para definir una ola de calor
    tiene_problemas_salud = True  # Reemplaza con True si el usuario tiene problemas de salud, False si no

    datos_clima = leer_datos_meteorologicos(nombre_archivo_json)
    etiquetas_ola_calor = etiquetar_ola_de_calor(datos_clima, umbral_temperatura_ola_calor)

    tipos_instalaciones_deportivas = {'PISCINA', 'ZONA VERDE', 'SENDA', 'CENTRO DE SALUD'}

    datos_instalaciones = []
    for tipo in tipos_instalaciones_deportivas:
        datos_instalaciones.extend(cargar_datos_instalaciones(nombre_archivo_sitios, tipo))

    recomendaciones_deportes = obtener_recomendaciones(
        datos_clima, etiquetas_ola_calor, datos_instalaciones, tiene_problemas_salud
    )

    # Imprime las recomendaciones
    print("Recomendaciones de deportes:")
    for deporte in recomendaciones_deportes:
        print(deporte)


Recomendaciones de deportes:
